In [1]:
# Get data from CSV
import csv
data = []
with open('data.csv') as csvfile:
    reader = csv.DictReader(csvfile)    
    for row in reader:
        text = " ".join(row.values())
        data.append(text)
        
data[0]

'No es necesaria. •\tRecopilación, procesamiento y análisis de información cualitativa y cuantitativa: La recopilación de información se aborda a través de la interacción con clientes, la revisión de documentación interna y la búsqueda de información pública. Posteriormente la misma es procesada en conjunto con los demás consultores con el objetivo de generar conclusiones que se traduzcan en acciones específicas.\n•\tAsistencia en el desarrollo de publicaciones: Periódicamente se realizan estudios relacionados a mercados financieros y de capitales que son publicados en diarios y revistas de reconocido prestigio. \n•\tAsistencia en el desarrollo de proyectos: En su mayoría, los mismos están dirigidos a identificar ámbitos de mejora en materia de mejores prácticas de Gobierno Corporativo, y eventualmente guiar su implementación. Ello aplica tanto para entidades públicas y privadas. Analista Junior de Finanzas y Gobierno Corporativo'

In [2]:
# Preprocessing the data

punctuations = ['•','/']
translator = str.maketrans("".join(punctuations),' '*len(punctuations))

proc_data = []
for text in data:
    text = text.lower()
    text = text.translate(translator)
    proc_data.append(text)
    
data = proc_data
data[0]

'no es necesaria.  \trecopilación, procesamiento y análisis de información cualitativa y cuantitativa: la recopilación de información se aborda a través de la interacción con clientes, la revisión de documentación interna y la búsqueda de información pública. posteriormente la misma es procesada en conjunto con los demás consultores con el objetivo de generar conclusiones que se traduzcan en acciones específicas.\n \tasistencia en el desarrollo de publicaciones: periódicamente se realizan estudios relacionados a mercados financieros y de capitales que son publicados en diarios y revistas de reconocido prestigio. \n \tasistencia en el desarrollo de proyectos: en su mayoría, los mismos están dirigidos a identificar ámbitos de mejora en materia de mejores prácticas de gobierno corporativo, y eventualmente guiar su implementación. ello aplica tanto para entidades públicas y privadas. analista junior de finanzas y gobierno corporativo'

In [4]:
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import StanfordPOSTagger
import multiprocessing
import signal

class TimeoutError (RuntimeError):
    pass

def handler (signum, frame):
    raise TimeoutError()

signal.signal (signal.SIGALRM, handler)

def divide_list(n_chunks, list_to_divide):
    chunk_size = len(list_to_divide)//n_chunks + 1
    chunks = [list_to_divide[i:i+chunk_size] for i in range(0, len(list_to_divide), chunk_size)]

    return chunks


spanish_postagger = StanfordPOSTagger('models/spanish.tagger', './stanford-postagger.jar')

terms_by_tag = {}
terms_by_tag['n'] = set()
terms_by_tag['v'] = set()
terms_by_tag['a'] = set()
terms_by_tag['c'] = set()
terms_by_tag['d'] = set()
terms_by_tag['f'] = set()
terms_by_tag['i'] = set()
terms_by_tag['p'] = set()
terms_by_tag['r'] = set()
terms_by_tag['s'] = set()
terms_by_tag['w'] = set()
terms_by_tag['z'] = set()

tags_by_term = {}

print("Inicia revision")
for idx, text in enumerate(data):
    if idx % 100 == 0:
        print(" Avance: ", idx, " Tam. terms.: ", len(terms_by_tag['n']) + len(terms_by_tag['v']))
    try:
        signal.alarm(4)
        words = word_tokenize(text)
        
        new_words = False
        for word in words:
            if word not in tags_by_term:
                new_words = True

        if new_words: 
            tag_terms = spanish_postagger.tag(words)
            for term in tag_terms:
                word = term[0]
                tag = term[1][0]
                
                terms_by_tag[tag].add(word)
                tags_by_term[word] = tag

    except TimeoutError as ex:
        print(" Excede tiempo en: ", idx)
    


/home/howl24/.virtualenvs/TextClassification/lib/python3.5/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


Inicia revision
 Avance:  0  Tam. terms.:  0
 Avance:  100  Tam. terms.:  1778
 Excede tiempo en:  119
 Excede tiempo en:  132
 Avance:  200  Tam. terms.:  2486
 Avance:  300  Tam. terms.:  3081
 Avance:  400  Tam. terms.:  3570
 Avance:  500  Tam. terms.:  4059
 Excede tiempo en:  525
 Avance:  600  Tam. terms.:  4462
 Excede tiempo en:  645
 Avance:  700  Tam. terms.:  4799
 Excede tiempo en:  730
 Avance:  800  Tam. terms.:  5157
 Excede tiempo en:  839
 Avance:  900  Tam. terms.:  5504
 Avance:  1000  Tam. terms.:  5745
 Excede tiempo en:  1001
 Excede tiempo en:  1087
 Excede tiempo en:  1097
 Avance:  1100  Tam. terms.:  6095
 Excede tiempo en:  1163
 Avance:  1200  Tam. terms.:  6401
 Avance:  1300  Tam. terms.:  6708
 Excede tiempo en:  1311
 Avance:  1400  Tam. terms.:  6929
 Excede tiempo en:  1418
 Excede tiempo en:  1459


LookupError: 

===========================================================================
NLTK was unable to find the java file!
Use software specific configuration paramaters or set the JAVAHOME environment variable.
===========================================================================

In [7]:
for tag in terms_by_tag:
    if "procesamiento" in terms_by_tag[tag]:
        print(tag)

n


In [98]:
vocab = {}
#vocab.update(dict.fromkeys(terms_by_tag['n'], 0))
#vocab.update(dict.fromkeys(terms_by_tag['v'], 0))
#vocab.update(dict.fromkeys(terms_by_tag['a'], 0))
len(vocab)

0

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize, ngram_range=(1, 4), min_df=15, max_df=0.5)
vectorizer.fit(data)
vect_vocab = vectorizer.get_feature_names()
vect_vocab = set(vect_vocab)
len(vect_vocab)



44011

In [99]:
#n #v #a
for phrase in vect_vocab:
    words = phrase.split()
    if len(words) == 1:
        try:
            if (tags_by_term[words[0]] == 'n' or
                tags_by_term[words[0]] == 'v' or
                tags_by_term[words[0]] == 'a'):
                #print(phrase)
                vocab[phrase] = 0                
        except:
            pass      
len(vocab)


4172

In [100]:
# n1 + s + n2
for phrase in vect_vocab:
    words = phrase.split()
    if len(words) == 3:
        try:
            if (tags_by_term[words[0]] == 'n' and
                tags_by_term[words[1]] == 's' and
                tags_by_term[words[2]] == 'n'):
                #print(phrase)
                vocab[phrase] = 0                
        except:
            pass      
len(vocab)

5760

In [101]:
# n + c + n

for phrase in vect_vocab:
    words = phrase.split()
    if len(words) == 3:
        try:
            if (tags_by_term[words[0]] == 'n' and
                tags_by_term[words[1]] == 'c' and
                tags_by_term[words[2]] == 'n'):
                vocab[phrase] = 0
                #print(phrase)
        except:
            pass
    
len(vocab)

6037

In [102]:
# n + c + n + a

for phrase in vect_vocab:
    words = phrase.split()
    if len(words) == 4:
        try:
            if (tags_by_term[words[0]] == 'n' and
                tags_by_term[words[1]] == 'c' and
                tags_by_term[words[2]] == 'n' and
                tags_by_term[words[3]] == 'a'):
                vocab[phrase] = 0
                #print(phrase)
        except:
            pass
len(vocab)  

6057

In [103]:
# v + s + n
for phrase in vect_vocab:
    words = phrase.split()
    if len(words) == 3:
        try:
            if (tags_by_term[words[0]] == 'v' and
                tags_by_term[words[1]] == 's' and                
                tags_by_term[words[2]] == 'n'):
                vocab[phrase] = 0
                #print(phrase)
        except:
            pass
len(vocab)

6126

In [104]:
# v + n
for phrase in vect_vocab:
    words = phrase.split()
    if len(words) == 2:
        try:
            if (tags_by_term[words[0]] == 'v' and                
                tags_by_term[words[1]] == 'n'):
                vocab[phrase] = 0
                #print(phrase)
        except:
            pass
len(vocab)

6380

In [105]:
# n + a
for phrase in vect_vocab:
    words = phrase.split()
    if len(words) == 2:
        try:
            if (tags_by_term[words[0]] == 'n' and                
                tags_by_term[words[1]] == 'a'):
                vocab[phrase] = 0
                #print(phrase)
        except:
            pass
len(vocab)


7401

In [106]:
# n + a + c + a
for phrase in vect_vocab:
    words = phrase.split()
    if len(words) == 2:
        try:
            if (tags_by_term[words[0]] == 'n' and
                tags_by_term[words[1]] == 'a' and
                tags_by_term[words[2]] == 'c' and
                tags_by_term[words[3]] == 'a'):
                vocab[phrase] = 0
                print(phrase)
        except:
            pass
len(vocab)


7401

In [107]:
import csv

with open('frases_15rep.csv', 'w') as csvfile:
    fn = ['Frase', 'Aceptar?']
    writer = csv.DictWriter(csvfile, fieldnames=fn)
    
    writer.writeheader()
    for row in vocab:
        writer.writerow({'Frase': row})
    











